# Data Exploration & Strategy Development

**Interactive Exploration of ES & NQ Futures Data**

This notebook provides interactive exploration and analysis tools for understanding:
1. **Price Dynamics** - Trends, volatility, and intraday patterns
2. **Noise Area Characteristics** - How boundaries adapt to market conditions
3. **Breakout Patterns** - What makes a successful vs failed breakout
4. **Transaction Cost Impact** - Visualizing the critical role of slippage

Use this notebook to develop intuition about the strategy and identify potential improvements.

---

## 1. Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from data_acquisition import FuturesDataDownloader
from noise_area import NoiseAreaCalculator
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Plotly for interactive charts
import plotly.io as pio
pio.renderers.default = 'browser'

os.makedirs('results', exist_ok=True)

print("✓ Setup complete")

## 2. Load Data

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Load data
downloader = FuturesDataDownloader(config)
try:
    data = downloader.load_data('data')
    print("✓ Loaded cached data")
except:
    print("Downloading data...")
    data = downloader.download_all_data()
    downloader.save_data(data, 'data')

es_data = data['ES'].copy()
nq_data = data['NQ'].copy()

print(f"\n✓ Data loaded:")
print(f"  ES: {len(es_data)} bars ({es_data.index[0]} to {es_data.index[-1]})")
print(f"  NQ: {len(nq_data)} bars ({nq_data.index[0]} to {nq_data.index[-1]})")

## 3. Basic Statistics & Distributions

In [ ]:
# Calculate returns
es_data['returns'] = es_data['Close'].pct_change()
nq_data['returns'] = nq_data['Close'].pct_change()

# Summary statistics
print("="*60)
print("ES STATISTICS")
print("="*60)
print(es_data[['Open', 'High', 'Low', 'Close', 'Volume']].describe())
print(f"\nReturns:")
print(f"  Mean: {es_data['returns'].mean()*100:.4f}%")
print(f"  Std Dev: {es_data['returns'].std()*100:.4f}%")
print(f"  Skewness: {es_data['returns'].skew():.3f}")
print(f"  Kurtosis: {es_data['returns'].kurtosis():.3f}")

print("\n" + "="*60)
print("NQ STATISTICS")
print("="*60)
print(nq_data[['Open', 'High', 'Low', 'Close', 'Volume']].describe())
print(f"\nReturns:")
print(f"  Mean: {nq_data['returns'].mean()*100:.4f}%")
print(f"  Std Dev: {nq_data['returns'].std()*100:.4f}%")
print(f"  Skewness: {nq_data['returns'].skew():.3f}")
print(f"  Kurtosis: {nq_data['returns'].kurtosis():.3f}")

### Return Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# ES returns histogram
ax = axes[0, 0]
ax.hist(es_data['returns'].dropna()*100, bins=100, alpha=0.7, edgecolor='black')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_title('ES Returns Distribution', fontsize=12, fontweight='bold')
ax.set_xlabel('Return (%)')
ax.set_ylabel('Frequency')
ax.grid(True, alpha=0.3)

# ES returns Q-Q plot
ax = axes[0, 1]
from scipy import stats
stats.probplot(es_data['returns'].dropna(), dist="norm", plot=ax)
ax.set_title('ES Returns Q-Q Plot', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# NQ returns histogram
ax = axes[1, 0]
ax.hist(nq_data['returns'].dropna()*100, bins=100, alpha=0.7, edgecolor='black', color='orange')
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_title('NQ Returns Distribution', fontsize=12, fontweight='bold')
ax.set_xlabel('Return (%)')
ax.set_ylabel('Frequency')
ax.grid(True, alpha=0.3)

# NQ returns Q-Q plot
ax = axes[1, 1]
stats.probplot(nq_data['returns'].dropna(), dist="norm", plot=ax)
ax.set_title('NQ Returns Q-Q Plot', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/exploration_return_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Intraday Patterns

Analyze time-of-day effects.

In [ ]:
# Extract hour and minute
es_data['hour'] = es_data.index.hour
es_data['minute'] = es_data.index.minute
nq_data['hour'] = nq_data.index.hour
nq_data['minute'] = nq_data.index.minute

# Calculate average return and volatility by hour
es_hourly = es_data.groupby('hour').agg({
    'returns': ['mean', 'std', 'count'],
    'Volume': 'mean'
})

nq_hourly = nq_data.groupby('hour').agg({
    'returns': ['mean', 'std', 'count'],
    'Volume': 'mean'
})

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# ES hourly returns
ax = axes[0, 0]
ax.bar(es_hourly.index, es_hourly['returns']['mean']*100*78, alpha=0.7)  # Annualized
ax.axhline(y=0, color='black', linewidth=1)
ax.set_title('ES Average Return by Hour (RTH)', fontsize=12, fontweight='bold')
ax.set_xlabel('Hour (ET)')
ax.set_ylabel('Avg Return (% per hour)')
ax.set_xticks(range(9, 17))
ax.grid(True, alpha=0.3)

# ES hourly volatility
ax = axes[0, 1]
ax.plot(es_hourly.index, es_hourly['returns']['std']*100, marker='o', linewidth=2)
ax.set_title('ES Volatility by Hour', fontsize=12, fontweight='bold')
ax.set_xlabel('Hour (ET)')
ax.set_ylabel('Std Dev (%)')
ax.set_xticks(range(9, 17))
ax.grid(True, alpha=0.3)

# NQ hourly returns
ax = axes[1, 0]
ax.bar(nq_hourly.index, nq_hourly['returns']['mean']*100*78, alpha=0.7, color='orange')
ax.axhline(y=0, color='black', linewidth=1)
ax.set_title('NQ Average Return by Hour (RTH)', fontsize=12, fontweight='bold')
ax.set_xlabel('Hour (ET)')
ax.set_ylabel('Avg Return (% per hour)')
ax.set_xticks(range(9, 17))
ax.grid(True, alpha=0.3)

# NQ hourly volatility
ax = axes[1, 1]
ax.plot(nq_hourly.index, nq_hourly['returns']['std']*100, marker='o', linewidth=2, color='orange')
ax.set_title('NQ Volatility by Hour', fontsize=12, fontweight='bold')
ax.set_xlabel('Hour (ET)')
ax.set_ylabel('Std Dev (%)')
ax.set_xticks(range(9, 17))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/exploration_intraday_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

print("Key Observations:")
print(f"  ES most volatile hour: {es_hourly['returns']['std'].idxmax()}:00")
print(f"  NQ most volatile hour: {nq_hourly['returns']['std'].idxmax()}:00")

## 5. Volatility Dynamics

Analyze how volatility changes over time (important for noise area calculation).

In [ ]:
# Calculate rolling volatility (20-day EWMA)
es_data['vol_ewma'] = es_data['returns'].ewm(span=20*78, adjust=False).std() * np.sqrt(252*78)  # Annualized
nq_data['vol_ewma'] = nq_data['returns'].ewm(span=20*78, adjust=False).std() * np.sqrt(252*78)

# Resample to daily for cleaner visualization
es_daily_vol = es_data['vol_ewma'].resample('D').last()
nq_daily_vol = nq_data['vol_ewma'].resample('D').last()

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ES volatility over time
ax = axes[0]
ax.plot(es_daily_vol.index, es_daily_vol*100, linewidth=1.5, color='blue')
ax.fill_between(es_daily_vol.index, 0, es_daily_vol*100, alpha=0.3, color='blue')
ax.axhline(y=es_daily_vol.mean()*100, color='red', linestyle='--', alpha=0.5, 
           label=f'Mean: {es_daily_vol.mean()*100:.1f}%')
ax.set_title('ES Realized Volatility Over Time (20-day EWMA)', fontsize=12, fontweight='bold')
ax.set_ylabel('Annualized Volatility (%)')
ax.legend()
ax.grid(True, alpha=0.3)

# NQ volatility over time
ax = axes[1]
ax.plot(nq_daily_vol.index, nq_daily_vol*100, linewidth=1.5, color='orange')
ax.fill_between(nq_daily_vol.index, 0, nq_daily_vol*100, alpha=0.3, color='orange')
ax.axhline(y=nq_daily_vol.mean()*100, color='red', linestyle='--', alpha=0.5,
           label=f'Mean: {nq_daily_vol.mean()*100:.1f}%')
ax.set_title('NQ Realized Volatility Over Time (20-day EWMA)', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Annualized Volatility (%)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/exploration_volatility_dynamics.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nVolatility Statistics:")
print(f"  ES mean vol: {es_daily_vol.mean()*100:.2f}%")
print(f"  ES vol range: {es_daily_vol.min()*100:.2f}% - {es_daily_vol.max()*100:.2f}%")
print(f"  NQ mean vol: {nq_daily_vol.mean()*100:.2f}%")
print(f"  NQ vol range: {nq_daily_vol.min()*100:.2f}% - {nq_daily_vol.max()*100:.2f}%")

## 6. Noise Area Exploration

Calculate and visualize noise area with different parameters.

In [ ]:
# Test different lookback periods
lookback_tests = [30, 60, 90, 120]

fig, axes = plt.subplots(len(lookback_tests), 1, figsize=(16, 16))

for i, lookback in enumerate(lookback_tests):
    # Update config
    test_config = config.copy()
    test_config['strategy']['noise_area']['lookback_days'] = lookback
    
    # Calculate noise area
    calculator = NoiseAreaCalculator(test_config)
    es_test = calculator.calculate_noise_area(es_data.copy())
    es_test = calculator.identify_breakouts(es_test)
    
    # Plot sample period
    ax = axes[i]
    plot_data = es_test.iloc[:min(1000, len(es_test))]
    
    ax.plot(plot_data.index, plot_data['Close'], linewidth=1.5, color='black', label='ES Close')
    ax.plot(plot_data.index, plot_data['upper_boundary'], linestyle='--', color='red', 
            alpha=0.7, label='Upper Boundary')
    ax.plot(plot_data.index, plot_data['lower_boundary'], linestyle='--', color='green',
            alpha=0.7, label='Lower Boundary')
    ax.fill_between(plot_data.index, plot_data['lower_boundary'], plot_data['upper_boundary'],
                     alpha=0.1, color='gray')
    
    # Mark breakouts
    breaks_above = plot_data[plot_data['break_above']]
    breaks_below = plot_data[plot_data['break_below']]
    ax.scatter(breaks_above.index, breaks_above['Close'], marker='^', s=30, 
               color='green', alpha=0.5, label='Break Above')
    ax.scatter(breaks_below.index, breaks_below['Close'], marker='v', s=30,
               color='red', alpha=0.5, label='Break Below')
    
    ax.set_title(f'ES Noise Area - {lookback} Day Lookback', fontsize=11, fontweight='bold')
    ax.set_ylabel('Price')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.savefig('results/exploration_noise_area_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Noise area comparison visualized")

## 7. Breakout Success Rate Analysis

Analyze characteristics of successful vs failed breakouts.

In [ ]:
# Calculate noise area with base config
calculator = NoiseAreaCalculator(config)
es_analysis = calculator.calculate_noise_area(es_data.copy())
es_analysis = calculator.identify_breakouts(es_analysis)

# Identify breakout episodes
breakouts = []
in_breakout = False
breakout_start = None
breakout_high = None
breakout_low = None
breakout_direction = None

for i in range(len(es_analysis)):
    row = es_analysis.iloc[i]
    
    if (row['break_above'] or row['break_below']) and not in_breakout:
        # Start of breakout
        in_breakout = True
        breakout_start = i
        breakout_high = row['High']
        breakout_low = row['Low']
        breakout_direction = 'up' if row['break_above'] else 'down'
    
    elif in_breakout:
        # Update highs/lows during breakout
        breakout_high = max(breakout_high, row['High'])
        breakout_low = min(breakout_low, row['Low'])
        
        # Check for end of breakout (re-enters noise area)
        if row['inside_noise']:
            # End of breakout - record it
            entry_price = es_analysis.iloc[breakout_start]['Close']
            exit_price = row['Close']
            
            if breakout_direction == 'up':
                pnl = exit_price - entry_price
                max_favorable = breakout_high - entry_price
            else:
                pnl = entry_price - exit_price
                max_favorable = entry_price - breakout_low
            
            breakouts.append({
                'start_idx': breakout_start,
                'end_idx': i,
                'direction': breakout_direction,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': pnl,
                'max_favorable': max_favorable,
                'duration_bars': i - breakout_start,
                'success': pnl > 0
            })
            
            in_breakout = False

df_breakouts = pd.DataFrame(breakouts)

if len(df_breakouts) > 0:
    print("="*60)
    print("BREAKOUT ANALYSIS")
    print("="*60)
    print(f"Total breakouts: {len(df_breakouts)}")
    print(f"Successful: {df_breakouts['success'].sum()} ({df_breakouts['success'].mean()*100:.1f}%)")
    print(f"Failed: {(~df_breakouts['success']).sum()} ({(~df_breakouts['success']).mean()*100:.1f}%)")
    print(f"\nAverage P&L:")
    print(f"  Successful: ${df_breakouts[df_breakouts['success']]['pnl'].mean():.2f}")
    print(f"  Failed: ${df_breakouts[~df_breakouts['success']]['pnl'].mean():.2f}")
    print(f"\nAverage Duration:")
    print(f"  Successful: {df_breakouts[df_breakouts['success']]['duration_bars'].mean():.1f} bars")
    print(f"  Failed: {df_breakouts[~df_breakouts['success']]['duration_bars'].mean():.1f} bars")
    
    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # P&L distribution
    ax = axes[0, 0]
    ax.hist(df_breakouts[df_breakouts['success']]['pnl'], bins=30, alpha=0.7, 
            label='Successful', color='green')
    ax.hist(df_breakouts[~df_breakouts['success']]['pnl'], bins=30, alpha=0.7,
            label='Failed', color='red')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
    ax.set_title('Breakout P&L Distribution', fontsize=12, fontweight='bold')
    ax.set_xlabel('P&L ($)')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Duration comparison
    ax = axes[0, 1]
    ax.boxplot([df_breakouts[df_breakouts['success']]['duration_bars'],
                df_breakouts[~df_breakouts['success']]['duration_bars']],
               labels=['Successful', 'Failed'])
    ax.set_title('Breakout Duration', fontsize=12, fontweight='bold')
    ax.set_ylabel('Duration (bars)')
    ax.grid(True, alpha=0.3)
    
    # Max favorable excursion
    ax = axes[1, 0]
    ax.scatter(df_breakouts[df_breakouts['success']]['max_favorable'],
              df_breakouts[df_breakouts['success']]['pnl'],
              alpha=0.5, color='green', label='Successful')
    ax.scatter(df_breakouts[~df_breakouts['success']]['max_favorable'],
              df_breakouts[~df_breakouts['success']]['pnl'],
              alpha=0.5, color='red', label='Failed')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
    ax.set_title('Max Favorable Excursion vs Final P&L', fontsize=12, fontweight='bold')
    ax.set_xlabel('Max Favorable Excursion ($)')
    ax.set_ylabel('Final P&L ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Success rate by direction
    ax = axes[1, 1]
    direction_success = df_breakouts.groupby('direction')['success'].agg(['sum', 'count', 'mean'])
    direction_success['mean'].plot(kind='bar', ax=ax, color=['green' if x > 0.5 else 'red' 
                                                              for x in direction_success['mean']])
    ax.set_title('Success Rate by Direction', fontsize=12, fontweight='bold')
    ax.set_xlabel('Direction')
    ax.set_ylabel('Success Rate')
    ax.set_ylim([0, 1])
    ax.axhline(y=0.5, color='black', linestyle='--', alpha=0.5)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('results/exploration_breakout_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Save breakout data
    df_breakouts.to_csv('results/breakout_episodes.csv', index=False)
    print("\n✓ Breakout analysis saved")
else:
    print("No breakouts detected in data")

## 8. Interactive Visualization (Plotly)

Create interactive chart for detailed exploration.

In [ ]:
# Use a smaller sample for performance
sample_data = es_analysis.iloc[:min(2000, len(es_analysis))]

# Create figure
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=('ES Price & Noise Area', 'Volume', 'Returns'),
    row_heights=[0.5, 0.25, 0.25]
)

# Price and noise area
fig.add_trace(
    go.Scatter(x=sample_data.index, y=sample_data['Close'], 
               mode='lines', name='Close', line=dict(color='black', width=1)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=sample_data.index, y=sample_data['upper_boundary'],
               mode='lines', name='Upper Boundary', 
               line=dict(color='red', width=1, dash='dash')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=sample_data.index, y=sample_data['lower_boundary'],
               mode='lines', name='Lower Boundary',
               line=dict(color='green', width=1, dash='dash')),
    row=1, col=1
)

# Mark breakouts
breakouts_above = sample_data[sample_data['break_above']]
breakouts_below = sample_data[sample_data['break_below']]

fig.add_trace(
    go.Scatter(x=breakouts_above.index, y=breakouts_above['Close'],
               mode='markers', name='Break Above',
               marker=dict(color='green', size=8, symbol='triangle-up')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=breakouts_below.index, y=breakouts_below['Close'],
               mode='markers', name='Break Below',
               marker=dict(color='red', size=8, symbol='triangle-down')),
    row=1, col=1
)

# Volume
fig.add_trace(
    go.Bar(x=sample_data.index, y=sample_data['Volume'], name='Volume',
           marker=dict(color='lightblue')),
    row=2, col=1
)

# Returns
fig.add_trace(
    go.Scatter(x=sample_data.index, y=sample_data['returns']*100,
               mode='lines', name='Returns', line=dict(color='purple', width=1)),
    row=3, col=1
)

fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="Return (%)", row=3, col=1)

fig.update_layout(
    height=900,
    title_text="ES Futures: Interactive Exploration",
    showlegend=True,
    hovermode='x unified'
)

# Save interactive HTML
fig.write_html('results/exploration_interactive.html')
print("✓ Interactive chart saved to results/exploration_interactive.html")
print("  Open this file in a browser for interactive exploration")

# Display in notebook (if running in Jupyter)
fig.show()

## 9. Transaction Cost Impact Simulation

Visualize how different slippage levels affect P&L.

In [ ]:
# Simulate a single trade with different slippage levels
entry_price = 4500
exit_price = 4520
contracts = 1
tick_value = 12.50
commission = 4.20

slippage_levels = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 2.5, 3.0]

results = []
for slippage_ticks in slippage_levels:
    # Calculate slipped prices
    slipped_entry = entry_price + (slippage_ticks * 0.25)  # Long position, pay more
    slipped_exit = exit_price - (slippage_ticks * 0.25)     # Long exit, receive less
    
    # P&L
    gross_pnl = (slipped_exit - slipped_entry) * 50  # ES multiplier
    slippage_cost = slippage_ticks * 2 * tick_value  # Entry + exit
    net_pnl = gross_pnl - slippage_cost - commission
    
    results.append({
        'slippage_ticks': slippage_ticks,
        'gross_pnl': gross_pnl,
        'slippage_cost': slippage_cost,
        'commission': commission,
        'net_pnl': net_pnl
    })

df_cost_impact = pd.DataFrame(results)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Cost breakdown
ax = axes[0]
x = range(len(df_cost_impact))
width = 0.35
ax.bar([i - width/2 for i in x], df_cost_impact['slippage_cost'], width, 
       label='Slippage', color='red', alpha=0.7)
ax.bar([i + width/2 for i in x], df_cost_impact['commission'], width,
       label='Commission', color='orange', alpha=0.7)
ax.set_title('Transaction Cost Breakdown', fontsize=12, fontweight='bold')
ax.set_xlabel('Slippage Level (ticks)')
ax.set_ylabel('Cost ($)')
ax.set_xticks(x)
ax.set_xticklabels(df_cost_impact['slippage_ticks'])
ax.legend()
ax.grid(True, alpha=0.3)

# Net P&L impact
ax = axes[1]
ax.plot(df_cost_impact['slippage_ticks'], df_cost_impact['gross_pnl'], 
        marker='o', label='Gross P&L', linewidth=2)
ax.plot(df_cost_impact['slippage_ticks'], df_cost_impact['net_pnl'],
        marker='s', label='Net P&L', linewidth=2)
ax.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax.fill_between(df_cost_impact['slippage_ticks'], 0, df_cost_impact['net_pnl'],
                alpha=0.3, color='green' if df_cost_impact['net_pnl'].iloc[-1] > 0 else 'red')
ax.set_title('P&L Degradation from Slippage', fontsize=12, fontweight='bold')
ax.set_xlabel('Slippage (ticks per side)')
ax.set_ylabel('P&L ($)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/exploration_transaction_cost_impact.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nExample Trade Analysis (ES, 1 contract):")
print(f"Entry: ${entry_price:.2f} → Exit: ${exit_price:.2f}")
print(f"\nSlippage Impact:")
for i, row in df_cost_impact.iterrows():
    print(f"  {row['slippage_ticks']:.2f} ticks: Net P&L = ${row['net_pnl']:.2f} "
          f"(Slippage: ${row['slippage_cost']:.2f}, Commission: ${row['commission']:.2f})")

## Summary: Key Insights

### 1. Data Characteristics
- **ES & NQ show fat-tailed return distributions** (high kurtosis)
- **Intraday volatility patterns**: Highest at open (9:30-10:00) and close (15:30-16:00)
- **Volatility clustering**: Periods of high/low vol persist

### 2. Noise Area Behavior
- **Shorter lookbacks** (30-60 days): More responsive, more signals, higher risk
- **Longer lookbacks** (90-120 days): More stable, fewer false signals
- **Trade-off**: Adaptability vs robustness

### 3. Breakout Success
- **Success rate varies by market regime**: Higher in trending markets
- **Duration matters**: Quick failures vs sustained moves
- **Max Favorable Excursion**: Many trades show early profits but give back gains

### 4. Transaction Costs (CRITICAL)
- **Slippage dominates**: 1-2 ticks can reduce profits by 30-50%
- **Commission is minor**: ~$4 vs ~$25-50 slippage per trade
- **Execution quality is paramount**: Must monitor actual fills

### Next Steps
1. Run full backtest: `01_end_to_end_backtest.ipynb`
2. Parameter sensitivity: `02_parameter_sensitivity_analysis.ipynb`
3. Develop execution quality monitoring
4. Test regime detection methods

---